## Get all unique swappers first tx

In [ ]:
import pandas as pd
import requests
import json
from math import ceil

# Load the swap.pkl dataset
df = pd.read_pickle('df_total_cut_11Apr_15Apr.pkl', compression='gzip')

# Specify the token address and pool address
token_address = '0xbe0dbc7d9a93e7f2a98b3218ac82e2a7e7ccb3e5'
# '0x0a13a5929e5f0ff0eaba4bd9e9512c91fce40280'
pool_address = '0x116de4c81d2828c838ec14d5cd9663fa8711065a'
# '0x01f4916850f37d262108a31aa7e1367441eba344'

# Filter the dataframe for swaps involving the current token and pool
filtered_swaps_xcorp = df[(df['buyCurrency.address'] == token_address) & (df['smartContract.address.address'] == pool_address)]

# Get all unique swapper addresses
unique_swappers_xorp = filtered_swaps_xcorp['transaction.txFrom.address'].unique()

# Print the result
url = 'https://graphql.bitquery.io/'

# Split the unique swappers into batches of 200 addresses
batches = [unique_swappers_xorp[i:i + 200] for i in range(0, len(unique_swappers_xorp), 200)]

# Create a DataFrame to store the results
results = []

# Iterate over the batches
for batch in batches:
    # Define the query
    batch_list = batch.tolist()
    query = {
        'query': '''
        query {
          ethereum {
            transfers {
              first_tx_time: minimum(of: time)
              first_block: minimum(of: block)
              first_tx_hash: minimum(of: time, get: tx_hash)
              first_tx_sender: minimum(of: time, get: sender)
              first_tx_currrency_address: minimum(of: time, get: currency_address)
              first_tx_currrency_symbol: minimum(of: time, get: currency_symbol)
              count
              receiver(
                receiver: {in: BATCH}
              ) {
                address
              }
            }
          }
        }
        '''.replace('BATCH', json.dumps(batch_list))
    }
    headers = {
    'Content-Type': 'application/json',
    'X-API-KEY': 'BQYvbQFK4xjlGgzv8IwQahp71H7Z4dDT'
    }
    
    # Send the request
    response = requests.post(url,headers=headers, json=query)

    # Parse the response
    data = json.loads(response.text)        

    # Extract the transfers and add them to the results
    results.extend(data['data']['ethereum']['transfers'])

# Write the results to a JSON file
with open('bq_xcorp_check_first_tx_data.json', 'w') as f:
    json.dump(results, f)




## Match unique with CEX addresses and get fresh

In [71]:
# import pandas as pd
# import json
# from datetime import datetime, timedelta
# import pytz

# Load the Dune dataset
dune_df = pd.read_csv('dune_default_cex.csv')

# Load the Bitquery results
with open('bitquery_check_first_tx_data.json', 'r') as f:
    bitquery_data = json.load(f)

bitquery_df_xorp = pd.DataFrame(bitquery_data)

# Extract the receiver.address from the nested JSON objects in the 'receiver' column
bitquery_df_xorp['receiver'] = bitquery_df_xorp['receiver'].apply(lambda x: x['address'])

# Filter the unique swappers that match with the first_tx_sender from Bitquery results
matched_swappers = bitquery_df_xorp[bitquery_df_xorp['first_tx_sender'].isin(dune_df['address'])]

# merge two datasets
merged_df_xai = pd.merge(matched_swappers, dune_df, left_on='first_tx_sender', right_on='address')

# Add a new column 'CEX' and fill it with 'name' from dune_df
merged_df_xai['CEX'] = merged_df_xai['name']


first_swaps = filtered_swaps_xcorp.groupby('transaction.txFrom.address').first().reset_index()

# Convert the 'receiver' column to just the address string. Run it only once
# bitquery_df_xorp['receiver'] = bitquery_df_xorp['receiver'].apply(lambda x: x['address'])

# Convert time columns to datetime and calculate the time difference
bitquery_df_xorp['first_tx_time'] = pd.to_datetime(bitquery_df_xorp['first_tx_time'])
first_swaps['block.timestamp.time'] = pd.to_datetime(first_swaps['block.timestamp.time'], utc=True)

# Merge 'bitquery_df_xorp' with 'merged_df_xai' to add 'CEX' column
bitquery_df_xorp = pd.merge(bitquery_df_xorp, merged_df_xai[['receiver', 'CEX']], on='receiver', how='left')

# Convert the 'first_tx_time' column to datetime and remove the timezone
bitquery_df_xorp['first_tx_time'] = bitquery_df_xorp['first_tx_time'].dt.tz_localize(None)


bitquery_df_xorp['first_swap_time'] = pd.to_datetime(first_swaps['block.timestamp.time']).dt.tz_localize(None)

# Calculate the time difference in hourss
bitquery_df_xorp['time_difference'] = (bitquery_df_xorp['first_swap_time'] - bitquery_df_xorp['first_tx_time']).dt.total_seconds() / 3600

# Filter addresses where the time difference is less than 24 hours
recent_addresses = bitquery_df_xorp[bitquery_df_xorp['time_difference'] < 24]

# Count the number of such addresses
address_count = len(recent_addresses)

print(f'The number of fresh addresses that were fullfilled from CEX within the last 24 hours is: {address_count}')



The number of fresh addresses that were fullfilled from CEX within the last 24 hours is: 57


## TOP 10 tokens from txt with the highest count of swaps

In [ ]:
# Load the list of token addresses from the tokens06_15.txt file
with open('tokens06_15.txt', 'r') as f:
    token_addresses = [line.strip() for line in f]

# Filter the dataframe to only include swaps involving the tokens from the list
df = df[df['buyCurrency.address'].isin(token_addresses)]

# Count the number of swaps associated with each token
token_swap_counts = df['buyCurrency.address'].value_counts()

# Get the top 10 tokens with the highest count of swaps
top_10_tokens = token_swap_counts.head(10)

print('The top 10 tokens with the highest count of swaps are:')
print(top_10_tokens)


In [ ]:
recent_addresses.to_excel('recent_addresses.xlsx', index=False)

In [ ]:
import pandas as pd
# Load the token addresses
with open('tokens06_15.txt', 'r') as f:
    token_addresses = f.read().splitlines()

# Initialize a dictionary to store the results
first_swappers = {}

df = pd.read_pickle('df_total_cut_11Apr_15Apr.pkl', compression='gzip')

# Iterate over each token address
for token in token_addresses:
    # Filter the dataframe for swaps involving the current token
    token_swaps = df[df['buyCurrency.address'] == token]
    # Sort by block height
    token_swaps = token_swaps.sort_values('block.height', ascending=True)
    # Check if there are any swaps
    if not token_swaps.empty:
        # The address of the first swapper is the 'txFrom.address' of the first row
        first_swapper = token_swaps.iloc[0]['transaction.txFrom.address']
        # Store the result
        first_swappers[token] = first_swapper

# Print the result
for token, swapper in first_swappers.items():
    print(f'The first address to swap with token {token} was {swapper}.')


In [ ]:
import pandas as pd
unpickled_df = pd.read_pickle('trades_with_freshcoin_uni2_06Apr_15Apr.pkl', compression='gzip')
